In [1]:
import os
import re
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import DirectoryLoader
import magic
import os
import nltk

In [2]:
import os

os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'


In [3]:

# PDF to text conversion functions
def convert_pdf_to_text(pdf_path):
    laparams = LAParams()
    text = extract_text(pdf_path, laparams=laparams)
    cleaned_text = clean_text(text)
    return cleaned_text

def clean_text(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    cleaned_text = cleaned_text.replace('\n', ' ')
    cleaned_text = cleaned_text.lower()
    return cleaned_text

def process_pdfs(input_dir, output_dir):
    for file in os.listdir(input_dir):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(input_dir, file)
            text = convert_pdf_to_text(pdf_path)

            output_file = os.path.join(output_dir, f"{os.path.splitext(file)[0]}.txt")
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(text)
                
# Set input and output directories
input_dir = 'C:/Users/visha/OneDrive/Desktop/Miracle Project/pdf/'
output_dir = 'C:/Users/visha/OneDrive/Desktop/Miracle Project/text/'

# Process all PDFs in the input directory
process_pdfs(input_dir, output_dir)

In [4]:
loader = DirectoryLoader('./text/', glob='**/*.txt')
documents = loader.load()


In [5]:
documents

[Document(page_content="personal finance is an essential part of our daily lives that impacts our financial well-being in the short and long term. however, it can be challenging to manage our finances effectively. here are some tips to help you improve your personal finance skills: first, create a budget. this involves identifying your income and expenses, and allocating your money to different categories. a budget helps you track your spending and make informed decisions about where to allocate your money. second, set financial goals for yourself. this could include saving for a down payment on a house, paying off debt, or investing for retirement. having clear goals helps you stay motivated and focused on your long-term financial success. third, reduce debt. debt can be a major barrier to achieving your financial goals. focus on paying it down as quickly as possible. consider consolidating high-interest debt, negotiating with creditors for lower interest rates, or finding ways to red

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [8]:
texts


[Document(page_content="personal finance is an essential part of our daily lives that impacts our financial well-being in the short and long term. however, it can be challenging to manage our finances effectively. here are some tips to help you improve your personal finance skills: first, create a budget. this involves identifying your income and expenses, and allocating your money to different categories. a budget helps you track your spending and make informed decisions about where to allocate your money. second, set financial goals for yourself. this could include saving for a down payment on a house, paying off debt, or investing for retirement. having clear goals helps you stay motivated and focused on your long-term financial success. third, reduce debt. debt can be a major barrier to achieving your financial goals. focus on paying it down as quickly as possible. consider consolidating high-interest debt, negotiating with creditors for lower interest rates, or finding ways to red

In [9]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
docsearch = Chroma.from_documents(texts, embeddings)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [10]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch)
query = "Why is credit score importnant?"
qa.run(query)

" Credit score is important because it plays a major role in your ability to obtain loans, credit cards, and other financial products. Lenders use credit scores to determine whether you are a reliable borrower, so it's important to regularly monitor your credit score and take steps to improve it if necessary."